In [702]:
import torch.nn as nn
import torch
import pandas as pd

import torch.optim as optim  
import torch.nn.functional as F  
from torch.utils.data import DataLoader, Dataset  

import nltk
import string
from torch.utils.data import TensorDataset, DataLoader

# Load data

In [71]:
mr_all = pd.read_csv('data/mr_all.csv')

In [76]:
mr_all.Name

'idk'

In [704]:
print(len(mr_all["text"][0]),"::::",mr_all["text"][0])

177 :::: the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .


# Preprocess

In [705]:
nltk.data.path.append("./data")
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))


In [706]:
def preprocess_text(text, remove_stopwords=True):
    text = text.lower()  # Lowercase
    text = "".join([ch for ch in text if ch not in string.punctuation])  # Remove punctuation
    tokens = text.split()  # Tokenize

    if remove_stopwords:
        tokens = [word for word in tokens if word not in stop_words]

    return tokens  # return as list of words

In [79]:
type(mr_all["text"])

pandas.core.series.Series

In [81]:
type(mr_all)

pandas.core.frame.DataFrame

In [707]:
mr_all["tokens"] = mr_all["text"].apply(preprocess_text, remove_stopwords=False)

In [708]:
print(len(mr_all["tokens"][0]),"::::",mr_all["tokens"][0])

30 :::: ['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'centurys', 'new', 'conan', 'and', 'that', 'hes', 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', 'jeanclaud', 'van', 'damme', 'or', 'steven', 'segal']


In [709]:
all_words = [word for tokens in mr_all["tokens"] for word in tokens]
print(f"Total tokens: {len(all_words)}, Unique words: {len(set(all_words))}")

Total tokens: 197296, Unique words: 20492


In [710]:
type(mr_all["tokens"])

pandas.core.series.Series

In [711]:
mr_all
all_words

['the',
 'rock',
 'is',
 'destined',
 'to',
 'be',
 'the',
 '21st',
 'centurys',
 'new',
 'conan',
 'and',
 'that',
 'hes',
 'going',
 'to',
 'make',
 'a',
 'splash',
 'even',
 'greater',
 'than',
 'arnold',
 'schwarzenegger',
 'jeanclaud',
 'van',
 'damme',
 'or',
 'steven',
 'segal',
 'the',
 'gorgeously',
 'elaborate',
 'continuation',
 'of',
 'the',
 'lord',
 'of',
 'the',
 'rings',
 'trilogy',
 'is',
 'so',
 'huge',
 'that',
 'a',
 'column',
 'of',
 'words',
 'cannot',
 'adequately',
 'describe',
 'cowriterdirector',
 'peter',
 'jacksons',
 'expanded',
 'vision',
 'of',
 'j',
 'r',
 'r',
 'tolkiens',
 'middleearth',
 'effective',
 'but',
 'tootepid',
 'biopic',
 'if',
 'you',
 'sometimes',
 'like',
 'to',
 'go',
 'to',
 'the',
 'movies',
 'to',
 'have',
 'fun',
 'wasabi',
 'is',
 'a',
 'good',
 'place',
 'to',
 'start',
 'emerges',
 'as',
 'something',
 'rare',
 'an',
 'issue',
 'movie',
 'thats',
 'so',
 'honest',
 'and',
 'keenly',
 'observed',
 'that',
 'it',
 'doesnt',
 'feel'

# Label Encoding

In [712]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
mr_all["label"] = le.fit_transform(mr_all["label"])  # Now 0 or 1

In [713]:
mr_all["label"]

0        1
1        1
2        1
3        1
4        1
        ..
10657    0
10658    0
10659    0
10660    0
10661    0
Name: label, Length: 10662, dtype: int64

In [714]:
from collections import Counter

word_counts = Counter(all_words)
vocab = {word: i+2 for i, (word, _) in enumerate(word_counts.most_common())}
vocab["<PAD>"] = 0
vocab["<UNK>"] = 1


In [715]:
vocab["<PAD>"], vocab["<UNK>"], len(vocab)


(0, 1, 20494)

In [716]:
vocab.get("not")

23

In [717]:
def tokens_to_ids(tokens, vocab):
    return [vocab.get(token, vocab["<UNK>"]) for token in tokens]

mr_all["token_ids"] = mr_all["tokens"].apply(lambda tokens: tokens_to_ids(tokens, vocab))


In [718]:
mr_all

,text,label,tokens,token_ids
0,the rock is destined to be the 21st century's ...,1,"[the, rock, is, destined, to, be, the, 21st, c...","[2, 644, 7, 2540, 6, 20, 2, 3243, 10291, 95, 7..."
1,"the gorgeously elaborate continuation of "" the...",1,"[the, gorgeously, elaborate, continuation, of,...","[2, 3244, 2070, 7137, 5, 2, 4500, 5, 2, 2847, ..."
2,effective but too-tepid biopic,1,"[effective, but, tootepid, biopic]","[673, 13, 10295, 1904]"
3,if you sometimes like to go to the movies to h...,1,"[if, you, sometimes, like, to, go, to, the, mo...","[38, 22, 246, 28, 6, 192, 6, 2, 67, 6, 36, 127..."
4,"emerges as something rare , an issue movie tha...",1,"[emerges, as, something, rare, an, issue, movi...","[1321, 12, 100, 377, 18, 1759, 19, 109, 37, 50..."
...,...,...,...,...
10657,"this picture is murder by numbers , and as eas...",0,"[this, picture, is, murder, by, numbers, and, ...","[16, 164, 7, 820, 24, 1317, 4, 12, 378, 6, 20,..."
10658,hilarious musical comedy though stymied by acc...,0,"[hilarious, musical, comedy, though, stymied, ...","[472, 1347, 55, 132, 20491, 24, 2912, 2254, 12..."
10659,"if you are into splatter movies , then you wil...",0,"[if, you, are, into, splatter, movies, then, y...","[38, 22, 30, 45, 20492, 67, 211, 22, 54, 326, ..."
10660,"a dull , simple-minded and stereotypical tale ...",0,"[a, dull, simpleminded, and, stereotypical, ta...","[3, 268, 4467, 4, 3752, 168, 5, 1911, 444, 4, ..."


In [719]:
import torch
from torch.nn.utils.rnn import pad_sequence

# Convert to list of tensors
sequence_tensors = [torch.tensor(seq) for seq in mr_all["token_ids"]]

# Pad
padded_seqs = pad_sequence(sequence_tensors, batch_first=True, padding_value=vocab["<PAD>"])
label_tensor = torch.tensor(mr_all["label"].values)


In [720]:
len(padded_seqs[0])

51

In [721]:
padded_seqs

tensor([[    2,   644,     7,  ...,     0,     0,     0],
        [    2,  3244,  2070,  ...,     0,     0,     0],
        [  673,    13, 10295,  ...,     0,     0,     0],
        ...,
        [   38,    22,    30,  ...,     0,     0,     0],
        [    3,   268,  4467,  ...,     0,     0,     0],
        [    2,  2274,  6800,  ...,     0,     0,     0]])

# dataset defintion

In [722]:
from torch.utils.data import Dataset

class ReviewDataset(Dataset):
    def __init__(self, padded_seqs, labels):
        self.seqs = padded_seqs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.seqs[idx], self.labels[idx]

In [723]:
from sklearn.model_selection import train_test_split

# Assume `padded_seqs` and `label_tensor` are torch.Tensors
X_train, X_test, y_train, y_test = train_test_split(
    padded_seqs, label_tensor, test_size=0.2, random_state=42
)


In [724]:
X_train 

tensor([[4004, 1169,  404,  ...,    0,    0,    0],
        [8402,  856,    2,  ...,    0,    0,    0],
        [  51,    2, 1070,  ...,    0,    0,    0],
        ...,
        [1478, 2758, 2233,  ...,    0,    0,    0],
        [  18, 1678, 2361,  ...,    0,    0,    0],
        [  10, 6946,    5,  ...,    0,    0,    0]])

In [725]:
train_dataset = ReviewDataset(X_train, y_train)
test_dataset = ReviewDataset(X_test, y_test)

In [726]:
train_dataset 

In [727]:
batch_size = 32
# train_dataset = TensorDataset(X_train, y_train)
# test_dataset  = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader  = DataLoader(test_dataset, batch_size=batch_size)

## Model


In [728]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        embedded = self.embedding(x)                          # [batch, seq_len, embed_dim]
        _, (hidden, _) = self.lstm(embedded)                  # hidden: [2, batch, hidden_dim]
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)   # [batch, hidden_dim * 2]
        output = self.dropout(hidden)
        return self.fc(output)                                # [batch, output_dim]


In [729]:
# model = LSTMClassifier(
#     vocab_size=len(vocab),
#     embedding_dim=128,
#     hidden_dim=256,
#     output_dim=1,
#     pad_idx=vocab["<PAD>"]
# )


In [730]:
import torch
import torch.nn as nn

class StackedBiLSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx, num_layers=2, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        
        # Because of bidirection, final hidden state will have shape [2 * hidden_dim]
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.embedding(x)  # [batch, seq_len, embed_dim]
        
        # Pass through LSTM: get hidden states from last layer
        _, (hidden, _) = self.lstm(embedded)  # hidden: [num_layers * 2, batch, hidden_dim]
        
        # Concatenate the final forward and backward hidden states from the top LSTM layer
        hidden_fwd = hidden[-2, :, :]  # Last forward layer
        hidden_bwd = hidden[-1, :, :]  # Last backward layer
        hidden_cat = torch.cat((hidden_fwd, hidden_bwd), dim=1)  # [batch, hidden_dim * 2]

        output = self.dropout(hidden_cat)
        return self.fc(output)  # [batch, output_dim]


In [731]:
model = StackedBiLSTMClassifier(
    vocab_size=len(vocab),
    embedding_dim=128,
    hidden_dim=256,
    output_dim=1,
    pad_idx=vocab["<PAD>"],
    num_layers=4,
    dropout=0.3
)

In [732]:
X_train[15].shape

torch.Size([51])

In [733]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


StackedBiLSTMClassifier(
  (embedding): Embedding(20494, 128, padding_idx=0)
  (lstm): LSTM(128, 256, num_layers=4, batch_first=True, dropout=0.3, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [734]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

model = model.to(device)
criterion = criterion.to(device)


In [735]:
def binary_accuracy(preds, y):
    rounded = torch.round(torch.sigmoid(preds))  # sigmoid to get 0–1
    correct = (rounded == y).float()
    return correct.sum() / len(correct)


In [736]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    epoch_loss = 0
    epoch_acc = 0

    for x, y in dataloader:
        x, y = x.to(device), y.to(device).float()

        optimizer.zero_grad()
        preds = model(x).squeeze(1)  # [batch]
        loss = criterion(preds, y)
        acc = binary_accuracy(preds, y)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)


def evaluate(model, dataloader, criterion):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0

    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device).float()
            preds = model(x).squeeze(1)
            loss = criterion(preds, y)
            acc = binary_accuracy(preds, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)


In [737]:
model.eval()

StackedBiLSTMClassifier(
  (embedding): Embedding(20494, 128, padding_idx=0)
  (lstm): LSTM(128, 256, num_layers=4, batch_first=True, dropout=0.3, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [748]:
EPOCHS = 15

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    #val_loss, val_acc = evaluate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
    #print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc*100:.2f}%\n")


Epoch 1:
  Train Loss: 0.0216 | Train Acc: 99.47%
Epoch 2:
  Train Loss: 0.0166 | Train Acc: 99.61%
Epoch 3:
  Train Loss: 0.0149 | Train Acc: 99.73%
Epoch 4:
  Train Loss: 0.0290 | Train Acc: 99.26%
Epoch 5:
  Train Loss: 0.0175 | Train Acc: 99.67%
Epoch 6:
  Train Loss: 0.0312 | Train Acc: 99.27%
Epoch 7:
  Train Loss: 0.0186 | Train Acc: 99.60%
Epoch 8:
  Train Loss: 0.0216 | Train Acc: 99.50%
Epoch 9:
  Train Loss: 0.0080 | Train Acc: 99.87%
Epoch 10:
  Train Loss: 0.0109 | Train Acc: 99.80%
Epoch 11:
  Train Loss: 0.0172 | Train Acc: 99.52%
Epoch 12:
  Train Loss: 0.0131 | Train Acc: 99.61%
Epoch 13:
  Train Loss: 0.0089 | Train Acc: 99.82%
Epoch 14:
  Train Loss: 0.0226 | Train Acc: 99.36%
Epoch 15:
  Train Loss: 0.0048 | Train Acc: 99.92%


In [749]:
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc*100:.2f}%")


Test Loss: 1.7711 | Test Acc: 72.24%


In [750]:
sentence = "I love this movie, it was not bad at all, as others say! The performance of the main actor was terrific, and the plot was nigger."
tokens = preprocess_text(sentence, remove_stopwords=False)
token_ids = tokens_to_ids(tokens, vocab)

In [751]:
token_ids

[41,
 85,
 16,
 19,
 11,
 92,
 23,
 82,
 31,
 34,
 12,
 589,
 245,
 2,
 149,
 5,
 2,
 1406,
 447,
 92,
 471,
 4,
 2,
 114,
 92,
 1]

In [752]:
input_tensor = torch.tensor(token_ids, dtype=torch.long).unsqueeze(0)  # shape: (1, seq_len)

In [753]:
input_tensor = input_tensor.to(device)
model.eval()

StackedBiLSTMClassifier(
  (embedding): Embedding(20494, 128, padding_idx=0)
  (lstm): LSTM(128, 256, num_layers=4, batch_first=True, dropout=0.3, bidirectional=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [754]:
with torch.no_grad():
    output = model(input_tensor)

In [755]:
output

tensor([[0.6735]], device='cuda:0')

In [756]:
prob = torch.sigmoid(output).item()
prob

0.6622800230979919

In [757]:
pred = int(prob >= 0.5)
pred

1

In [760]:
ng20_all = pd.read_csv("./data/ng_20_all.csv")

In [761]:
ng20_all

,text,label,label_text
0,I was wondering if anyone out there could enli...,7,rec.autos
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware
3,\nDo you have Weitek's address/phone number? ...,1,comp.graphics
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space
...,...,...,...
18841,"\n Henry, if I read you correctly, you may b...",14,sci.space
18842,"about\nthem on\n\nActually, I thought Macs wer...",4,comp.sys.mac.hardware
18843,"I sent a version of this post out a while ago,...",9,rec.sport.baseball
18844,I have this kit which includes the following :...,6,misc.forsale


In [763]:
ng20_all

,text,label,label_text
0,I was wondering if anyone out there could enli...,7,rec.autos
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware
3,\nDo you have Weitek's address/phone number? ...,1,comp.graphics
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space
...,...,...,...
18841,"\n Henry, if I read you correctly, you may b...",14,sci.space
18842,"about\nthem on\n\nActually, I thought Macs wer...",4,comp.sys.mac.hardware
18843,"I sent a version of this post out a while ago,...",9,rec.sport.baseball
18844,I have this kit which includes the following :...,6,misc.forsale


In [762]:
ng20_all["tokens"] = ng20_all["text"].apply(preprocess_text, remove_stopwords=False)

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
type(ng20_all["text"])

pandas.core.series.Series

In [70]:
# what data type is column "text" in ng20_all

i = 0
for row in ng20_all[0:5]["text"]:

    if not type(row) == str:
        print(i, row)
    i+= 1

NameError: name 'ng20_all' is not defined